In [1]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.webservice import AciWebservice 
from azureml.core.model import Model
from azureml.core import Workspace, Experiment


# ws = Workspace.get(name="capstone")
ws = Workspace.from_config()
# exp = Experiment(workspace=ws, name="udacity-project-hyperparam-bayes")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: capstone
Azure region: westeurope
Subscription id: 72f46e0e-1451-4b79-92cd-fc8f7797bda7
Resource group: test-Jesse


In [2]:
model = Model(ws, name='automl')
service = Model.deploy(workspace=ws,
                        name = 'automl',
                        models= [model],
                        inference_config=inference_config,
                        deployment_config=aci_config
)

service.wait_for_deployment(True)
print(service.state)
print("scoring URI: " + service.scoring_uri)

NameError: name 'inference_config' is not defined

In [ ]:
import pandas as pd
import json
import requests 

test_url = "https://raw.githubusercontent.com/jvanelteren/housing/master/datasets/final_test.csv"
test_x = pd.read_csv(test_url)
data = test_x.to_json()

headers = {'Content-Type':'application/json'}

test_data = json.dumps(data)
data = json.loads(data)
test_x = data
response = requests.post(service.scoring_uri, data=test_data, headers=headers)
print(response.status_code)
print(response.elapsed)
print(response.json())
